In [13]:
import json
from collections import defaultdict
from sqlalchemy import create_engine
from django.core.management.base import BaseCommand
from django.conf import settings
from appcreator.import_utils import run_import

# imports for custom things
import json
from tqdm import tqdm
import pandas as pd
from django.core.serializers.json import DjangoJSONEncoder
from archiv.models import KeyWord, Stelle


dbc = settings.LEGACY_DB_CONNECTION
db_connection_str = f"mysql+pymysql://{dbc['USER']}:{dbc['PASSWORD']}@{dbc['HOST']}/{dbc['NAME']}"
db_connection = create_engine(db_connection_str)

In [4]:
for x in KeyWord.objects.all():
    x.stichwort = x.stichwort.strip()
    x.save()

In [5]:
query = "SELECT * FROM stelle"
df = pd.read_sql(query, con=db_connection)

In [7]:
stichwort_list = [x for x in list(df.keys()) if x.startswith('sstich')]
ambique = {}
no_match = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    if row['sid']:
        try:
            item = Stelle.objects.get(legacy_pk=row['sid'])
        except:
            continue
        for st in stichwort_list:
            if row[st] != "":
                try:
                    lookup = row[st].strip()
                except AttributeError:
                    continue
                kws = KeyWord.objects.filter(stichwort=lookup)
                if len(kws) == 1:
                    item.key_word.add(*kws)
                elif len(kws) == 0:
                    no_match[f"{item}"] = lookup
                else:
                    ambique[f"{item}"] = lookup

100%|██████████| 4661/4661 [00:52<00:00, 88.60it/s] 


In [10]:
v = defaultdict(list)

In [11]:
for key, value in sorted(no_match.items()):
    v[value].append(key)

In [12]:
with open('no_match.json', 'w') as fp:
    json.dump(v, fp, indent=True)

In [14]:
with open('no_match.json') as fp:
    no_match = json.load(fp)

In [15]:
no_match_in_variants = []
matches = defaultdict(list)
ambique = defaultdict(list)
for key, value in v.items():
    potential_matches = KeyWord.objects.filter(varianten__icontains=key)
    if potential_matches.count() == 0:
        no_match_in_variants.append({key: value})
    elif potential_matches.count() == 1:
        matches[key] = potential_matches
    else:
        ambique[key] = potential_matches

In [16]:
results = [
    ["no_match_in_variants", no_match_in_variants],
    ["matches", matches],
    ["ambique", ambique]
]

In [17]:
for x in results:
    print(f"{x[0]}: {len(x[1])}")

no_match_in_variants: 37
matches: 92
ambique: 16


In [18]:
ambique, [list(x.keys())[0] for x in no_match_in_variants]

(defaultdict(list,
             {'Franci': <QuerySet [<KeyWord: Francia>, <KeyWord: francicus>, <KeyWord: Francigenus>, <KeyWord: Francus>]>,
              'Graeci': <QuerySet [<KeyWord: Graecia>, <KeyWord: Graecus>]>,
              'Gothus': <QuerySet [<KeyWord: Gothi>, <KeyWord: Wisigothi>]>,
              'Romani': <QuerySet [<KeyWord: civis Romanus>, <KeyWord: Romania>, <KeyWord: Romanus>]>,
              'gentes': <QuerySet [<KeyWord: gens>, <KeyWord: gens Dei>]>,
              'Alamanni': <QuerySet [<KeyWord: Alamannia>, <KeyWord: Alamannus>]>,
              'Scotti': <QuerySet [<KeyWord: Scottia>, <KeyWord: Scotticus>, <KeyWord: Scottus>]>,
              'populi': <QuerySet [<KeyWord: gens populi>, <KeyWord: populi gentium>, <KeyWord: populi nationum>, <KeyWord: populus>, <KeyWord: populus christianus>, <KeyWord: populus Dei>]>,
              'Aquitani': <QuerySet [<KeyWord: Aquitania>, <KeyWord: Aquitanus>]>,
              'Hispani': <QuerySet [<KeyWord: Hispania>, <KeyWord: Hi

In [19]:
no_match_in_variants = []
matches = defaultdict(list)
ambique = defaultdict(list)
for key, value in v.items():
    potential_matches = KeyWord.objects.filter(varianten__contains=key)
    if potential_matches.count() == 0:
        no_match_in_variants.append({key: value})
    elif potential_matches.count() == 1:
        matches[key] = potential_matches
    else:
        ambique[key] = potential_matches

In [20]:
results = [
    ["no_match_in_variants", no_match_in_variants],
    ["matches", matches],
    ["ambique", ambique]
]

In [21]:
for x in results:
    print(f"{x[0]}: {len(x[1])}")

no_match_in_variants: 48
matches: 83
ambique: 14


In [22]:
ambique, [list(x.keys())[0] for x in no_match_in_variants]

(defaultdict(list,
             {'Franci': <QuerySet [<KeyWord: Francia>, <KeyWord: Francigenus>, <KeyWord: Francus>]>,
              'Graeci': <QuerySet [<KeyWord: Graecia>, <KeyWord: Graecus>]>,
              'Romani': <QuerySet [<KeyWord: civis Romanus>, <KeyWord: Romania>, <KeyWord: Romanus>]>,
              'gentes': <QuerySet [<KeyWord: gens>, <KeyWord: gens Dei>]>,
              'Alamanni': <QuerySet [<KeyWord: Alamannia>, <KeyWord: Alamannus>]>,
              'Scotti': <QuerySet [<KeyWord: Scottia>, <KeyWord: Scotticus>, <KeyWord: Scottus>]>,
              'populi': <QuerySet [<KeyWord: gens populi>, <KeyWord: populi gentium>, <KeyWord: populi nationum>, <KeyWord: populus>, <KeyWord: populus christianus>, <KeyWord: populus Dei>]>,
              'Aquitani': <QuerySet [<KeyWord: Aquitania>, <KeyWord: Aquitanus>]>,
              'Hispani': <QuerySet [<KeyWord: Hispania>, <KeyWord: Hispaniensis>, <KeyWord: Hispanus>]>,
              'Frisia': <QuerySet [<KeyWord: Fresia>, <KeyWord: